In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas

In [3]:
url = "https://www.melon.com/chart/index.htm"
headers = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"}

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
titles = soup.find_all("div", {"class": "ellipsis rank01"}) 
singers = soup.find_all("div", {"class": "ellipsis rank02"}) 
albums = soup.find_all("div", {"class": "ellipsis rank03"})

In [5]:
for t, s, a in zip(titles, singers, albums):
    print(t.find('a').text, end=' ')
    print(s.find('span', {"class": "checkEllipsis"}).text, end=' ')
    print(a.find('a').text)


바라만 본다 MSG워너비(M.O.M) MSG워너비 1집
Next Level aespa Next Level
Butter 방탄소년단 Butter
나를 아는 사람 MSG워너비(정상동기) MSG워너비 1집
헤픈 우연 헤이즈 (Heize) HAPPEN
치맛바람 (Chi Mat Ba Ram) 브레이브걸스 Summer Queen
Dun Dun Dance 오마이걸 (OH MY GIRL) Dear OHMYGIRL
Peaches (Feat. Daniel Caesar & Giveon) Justin Bieber Justice
신호등 이무진 신호등
롤린 (Rollin') 브레이브걸스 Rollin'
Weekend 태연 (TAEYEON) Weekend
Alcohol-Free TWICE (트와이스) Taste of Love
라일락 아이유 IU 5th Album 'LILAC'
ASAP STAYC(스테이씨) STAYDOM
Dynamite 방탄소년단 Dynamite (DayTime Version)
비와 당신 이무진 슬기로운 의사생활 시즌2 OST Part 1
안녕 (Hello) 조이 (JOY) 안녕 (Hello) - Special Album
운전만해 (We Ride) 브레이브걸스 We Ride
Celebrity 아이유 IU 5th Album 'LILAC'
하루만 더 빅마마 하루만 더
상상더하기 MSG워너비 TOP 8 (별루지, 김정수, 강창모, 정기석, 이동휘, 이상이, 박재정, 원슈타인) MSG워너비 TOP 8 경연곡
사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco) 호미들 사이렌 Remix
Timeless SG 워너비 Sg Wanna Be+
상상더하기 라붐 (LABOUM) Fresh Adventure
내 손을 잡아 아이유 최고의 사랑 OST Part.4
멜로디 ASH ISLAND 멜로디
비 오는 날 듣기 좋은 노래 (Feat. Colde) 에픽하이 (EPIK HIGH) 비 오는 날 듣기 좋은 노래
밝게 빛나는 별이 되어 비춰줄게 송이한 밝게 빛나는 별이 되어

## Link로 접속해서 crawling하기

In [6]:
url = "https://www.melon.com/chart/index.htm"
headers = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"}

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [7]:
table = soup.find("table", {"border": 1})

In [8]:
first_song = table.find_all("tr")[1]['data-song-no']
print(first_song)

33625988


In [9]:
song_url = f"https://www.melon.com/song/detail.htm?songId={first_song}"

song_request = requests.get(song_url, headers=headers)
song_page = BeautifulSoup(song_request.text, 'html.parser')

In [10]:
song_title = song_page.find("div", {"class": "song_name"})
song_title.contents[2].strip()

'바라만 본다'

In [11]:
song_artist = song_page.find("a", {"class": "artist_name"}).text
song_artist

'MSG워너비(M.O.M)'

In [12]:
song_like = song_page.find("div", {"class":"button d_song_like"}).find("span", {"class": "cnt"}).text
song_like.strip()

'총건수\r\n\t\t\t\t\t\t\t\t\t\t\t0'

## instagram 좋아요수 크롤링 시도

In [15]:
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
from time import sleep

In [16]:
driver = webdriver.Chrome('chromedriver')

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
instagram_url = "https://www.instagram.com/p/CQnUq6bBjuv/"

driver.get(instagram_url)
instagram_page = driver.page_source
instagram_soup = BeautifulSoup(instagram_page, 'lxml')

In [ ]:
instagram_soup.find_all(attrs={'class':'Nm9Fw'})[0].text

## Selenium을 이용한 melon 페이지 크롤링

In [ ]:
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import datetime

import pandas as pd
from tqdm import tqdm

In [ ]:
## 멜론 실시간 차트가 보여지는 url, 사파리로 접속한 header
url = "https://www.melon.com/chart/index.htm"
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"}

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
## 크롤링을 실행할 드라이버
driver = webdriver.Chrome('chromedriver')

In [ ]:
def get_table(soup):
    table = soup.find("table", {"border": 1})
    song_ids = [e['data-song-no'] for e in table.find_all("tr")[1:]]
    return song_ids


In [ ]:
class song_page():
    
    def __init__(self, id):
        self.id = id

    def get_page(self):
        url = f"https://www.melon.com/song/detail.htm?songId={self.id}"
        self.url = url

        driver.get(url)
        song_page = driver.page_source
        song_soup = BeautifulSoup(song_page, 'lxml')
        self.soup = song_soup

    def get_name(self):
        self.name = self.soup.find(attrs={'class':'song_name'}).contents[2].strip()

    def get_artist(self):
        self.artist = self.soup.find(attrs={'class':'artist_name'}).text

    def get_album(self):
        self.album = self.soup.find(attrs={'class':'list'}).contents[3].text

    def get_released(self):
        self.released = self.soup.find(attrs={'class':'list'}).contents[7].text

    def get_likes(self):
        self.likes = self.soup.find(attrs={'class':'cnt'}).text

    def get_comments(self):
        self.comments = self.soup.find(attrs={'id':'revCnt'}).text

    def get_now(self):
        self.now = datetime.datetime.now()

    def set_attributes(self):
        self.get_page()

        self.get_name()
        self.get_artist()
        self.get_album()
        self.get_released()
        self.get_likes()
        self.get_comments()
        self.get_now()

    def get_attributes(self):
        return (self.id,
                self.name, 
                self.artist,
                self.album,
                self.released,
                self.likes,
                self.comments,
                self.now)


In [ ]:
table = get_table(soup)
song_info = []

for song_id in tqdm(table):

    page = song_page(song_id)
    page.set_attributes()
    song_info.append(page.get_attributes())

In [ ]:
df = pd.DataFrame(song_info, 
                  columns=['id', 'name', 'artist', 'album', 'released date', 'likes', 'comments', 'time(now)'])

In [ ]:
df

### for test

In [ ]:
table = get_table(soup)
page = song_page(table[0])
page.set_attributes()
print(page.get_attributes())